In [1]:
import os
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor, as_completed
from matplotlib.backends.backend_pdf import PdfPages

# Create a folder for today's date
today = datetime.today().strftime('%Y-%m-%d')
data_folder = f"data/{today}"
output_folder = f"output/{today}"
os.makedirs(data_folder, exist_ok=True)
os.makedirs(output_folder, exist_ok=True)

# Function to download or load cached data
def get_data(ticker):
    file_path = f"{data_folder}/{ticker}.csv"
    
    # Check if file already exists
    if os.path.exists(file_path):
        print(f"Loading {ticker} data from {file_path}")
        try:
            return pd.read_csv(file_path, index_col=0, parse_dates=True)
        except Exception as e:
            print(f"Error loading cached file for {ticker}: {e}")
            return None
    else:
        print(f"Downloading {ticker} data and saving to {file_path}")
        try:
            df = yf.download(ticker, period='5y')
            if not df.empty:
                df.to_csv(file_path)
                return df
            else:
                print(f"Downloaded data for {ticker} is empty, skipping.")
                return None
        except Exception as e:
            print(f"Error downloading {ticker}: {e}")
            return None

# List of ETFs and Commodities
assets = ['SPY', 'USO']

commodities = ['CL=F', 'BZ=F']

# Combine assets and commodities
#all_assets = assets + commodities
#all_assets = current_assets

all_assets = [ 'SPY', 'IWM', 'QQQ' ]

all_assets_Original = [
    'SPY', 'USO', 'GC=F', 'TLT', 'XLU', 'XLK', 'XLY', 'DBC', 'EEM', 'ZROZ', 
    'IEF', 'XLP', 'XLRE', 'PDBC', 'GSG', 'DBA', 'JJM', 'COPX', 'TIP', 'UUP', 
    'ITB', 'EFA', 'VWO', 'XLC', 'XLV', 'XLF', 'XLI', 'XLB', 'XLE', 'XLRE', 
    'XLP', 'XLY', 'XLK', 'XLU', 'FXE', 'FXY', 'FXB', 'FXC', 'FXA', 'FXF', 'CNY',
    'CL=F', 'BZ=F', 'NG=F', 'RB=F', 'HO=F', 'GC=F', 'SI=F', 'HG=F', 
    'PL=F', 'PA=F', 'ALI=F', 'ZC=F', 'ZW=F', 'ZS=F', 'KC=F', 'SB=F', 
    'CT=F', 'CC=F', 'LB=F', 'LE=F', 'HE=F', 'GF=F'
]


# Function to download or load data for all assets in parallel
def download_data_parallel(assets):
    df_dict = {}
    failed_tickers = []
    with ThreadPoolExecutor(max_workers=1) as executor:
        future_to_ticker = {executor.submit(get_data, asset): asset for asset in assets}
        for future in as_completed(future_to_ticker):
            asset = future_to_ticker[future]
            try:
                df = future.result()
                if df is not None and 'Adj Close' in df.columns:
                    df_dict[asset] = df['Adj Close']
                else:
                    print(f"Data for {asset} is invalid or missing 'Adj Close', skipping.")
                    failed_tickers.append(asset)
            except Exception as e:
                print(f"Error processing {asset}: {e}")
                failed_tickers.append(asset)
    if failed_tickers:
        print(f"Failed to download data for: {', '.join(failed_tickers)}")
    return df_dict

df_dict = download_data_parallel(all_assets)

# Ensure we have valid data before continuing
if len(df_dict) == 0:
    print("No valid data was downloaded, exiting...")
else:
    # Calculate percentage change for all assets
    pct_change_dict = {}
    for asset, df in df_dict.items():
        if not df.empty:
            pct_change = df.pct_change().dropna()
            if not pct_change.empty:
                pct_change_dict[asset] = pct_change

    # Convert to DataFrame for easier handling
    combined_pct_change_df = pd.DataFrame(pct_change_dict).dropna()

    # Check if the combined DataFrame is still valid
    if combined_pct_change_df.empty:
        print("No valid percentage change data, exiting...")
    else:
        # Save all figures into a single PDF
        with PdfPages(f"{output_folder}/combined_plots.pdf") as pdf:

            # 1. Pair Plot with KDE
            g = sns.pairplot(combined_pct_change_df, diag_kind='kde')
            g.fig.suptitle('Pair Plot of Percentage Changes Across All Assets', y=1.02)
            pdf.savefig(g.fig)  # Save the figure to the PDF
            plt.close(g.fig)

            # 2. Correlation Heatmap
            def plot_correlation_heatmap(combined_pct_change_df):
                corr_matrix = combined_pct_change_df.corr()
                plt.figure(figsize=(16, 12))
                sns.heatmap(corr_matrix, annot=True, cmap='RdBu', center=0, 
                            annot_kws={"size": 8}, fmt='.2f', cbar_kws={"shrink": 0.8})
                plt.xticks(rotation=45, ha="right", fontsize=8)
                plt.yticks(fontsize=8)
                plt.title('Correlation Heatmap of Percentage Changes Across All Assets')
                pdf.savefig()  # Save the heatmap to the PDF
                plt.close()

            plot_correlation_heatmap(combined_pct_change_df)

            # 3. Time Series Overlay
            def plot_time_series_grid(combined_pct_change_df, assets, num_cols=3):
                num_plots = len(assets) * (len(assets) - 1) // 2
                num_rows = (num_plots // num_cols) + 1
                fig, axes = plt.subplots(num_rows, num_cols, figsize=(15, 10))
                axes = axes.flatten()
                plot_idx = 0
                for i, asset1 in enumerate(assets):
                    for asset2 in assets[i+1:]:
                        if asset1 in combined_pct_change_df.columns and asset2 in combined_pct_change_df.columns:
                            axes[plot_idx].plot(combined_pct_change_df.index, combined_pct_change_df[asset1], label=f'{asset1}', color='blue')
                            axes[plot_idx].plot(combined_pct_change_df.index, combined_pct_change_df[asset2], label=f'{asset2}', color='orange')
                            axes[plot_idx].set_title(f'{asset1} vs {asset2}')
                            axes[plot_idx].legend()
                            plot_idx += 1
                            if plot_idx >= len(axes):
                                break
                    if plot_idx >= len(axes):
                        break
                plt.tight_layout()
                pdf.savefig(fig)  # Save to the PDF
                plt.close(fig)

            plot_time_series_grid(combined_pct_change_df, all_assets)

            # 4. Ratio Analysis
            def plot_ratio_grid(df_dict, assets, num_cols=3):
                num_plots = len(assets) * (len(assets) - 1) // 2
                num_rows = (num_plots // num_cols) + 1
                fig, axes = plt.subplots(num_rows, num_cols, figsize=(15, 10))
                axes = axes.flatten()
                plot_idx = 0
                for i, asset1 in enumerate(assets):
                    for asset2 in assets[i+1:]:
                        if asset1 in df_dict and asset2 in df_dict:
                            ratio = df_dict[asset1] / df_dict[asset2]
                            ratio.plot(ax=axes[plot_idx])
                            axes[plot_idx].set_title(f'{asset1}/{asset2}')
                            plot_idx += 1
                            if plot_idx >= len(axes):
                                break
                    if plot_idx >= len(axes):
                        break
                plt.tight_layout()
                pdf.savefig(fig)  # Save to the PDF
                plt.close(fig)

            plot_ratio_grid(df_dict, all_assets)

            # 5. Rolling Correlation Plot
            def plot_rolling_corr_grid(combined_pct_change_df, assets, num_cols=3):
                num_plots = len(assets) * (len(assets) - 1) // 2
                num_rows = (num_plots // num_cols) + 1
                fig, axes = plt.subplots(num_rows, num_cols, figsize=(15, 10))
                axes = axes.flatten()
                plot_idx = 0
                for i, asset1 in enumerate(assets):
                    for asset2 in assets[i+1:]:
                        if asset1 in combined_pct_change_df.columns and asset2 in combined_pct_change_df.columns:
                            rolling_corr = combined_pct_change_df[asset1].rolling(window=30).corr(combined_pct_change_df[asset2])
                            rolling_corr.plot(ax=axes[plot_idx])
                            axes[plot_idx].set_title(f'{asset1} vs {asset2} (Rolling Corr)')
                            plot_idx += 1
                            if plot_idx >= len(axes):
                                break
                    if plot_idx >= len(axes):
                        break
                plt.tight_layout()
                pdf.savefig(fig)  # Save to the PDF
                plt.close(fig)

            plot_rolling_corr_grid(combined_pct_change_df, all_assets)

            # 6. Ratio Comparisons
            def plot_ratio_comparisons_grid(df_dict, assets, num_cols=3):
                num_plots = len(assets) * (len(assets) - 1) // 2
                num_rows = (num_plots // num_cols) + 1
                fig, axes = plt.subplots(num_rows, num_cols, figsize=(15, 10))
                axes = axes.flatten()
                plot_idx = 0
                for i, asset1 in enumerate(assets):
                    for asset2 in assets[i+1:]:
                        if asset1 in df_dict and asset2 in df_dict:
                            ratio = df_dict[asset1] / df_dict[asset2]
                            ratio.plot(ax=axes[plot_idx])
                            axes[plot_idx].set_title(f'{asset1} / {asset2}')
                            plot_idx += 1
                            if plot_idx >= len(axes):
                                break
                    if plot_idx >= len(axes):
                        break
                plt.tight_layout()
                pdf.savefig(fig)  # Save to the PDF
                plt.close(fig)

            plot_ratio_comparisons_grid(df_dict, all_assets)

Loading SPY data from data/2024-11-24/SPY.csv


[*********************100%%**********************]  1 of 1 completed


[*********************100%%**********************]  1 of 1 completed


In [2]:
import os
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor, as_completed
from matplotlib.backends.backend_pdf import PdfPages

# Create a folder for today's date
today = datetime.today().strftime('%Y-%m-%d')
data_folder = f"data/{today}"
output_folder = f"output/{today}"
os.makedirs(data_folder, exist_ok=True)
os.makedirs(output_folder, exist_ok=True)

# Function to download or load cached data
def get_data(ticker):
    file_path = f"{data_folder}/{ticker}.csv"
    
    # Check if file already exists
    if os.path.exists(file_path):
        print(f"Loading {ticker} data from {file_path}")
        try:
            return pd.read_csv(file_path, index_col=0, parse_dates=True)
        except Exception as e:
            print(f"Error loading cached file for {ticker}: {e}")
            return None
    else:
        print(f"Downloading {ticker} data and saving to {file_path}")
        try:
            df = yf.download(ticker, period='5y')
            if not df.empty:
                df.to_csv(file_path)
                return df
            else:
                print(f"Downloaded data for {ticker} is empty, skipping.")
                return None
        except Exception as e:
            print(f"Error downloading {ticker}: {e}")
            return None

# List of assets
all_assets_orig = [
    'SPY', 'USO', 'GC=F', 'TLT', 'XLU', 'XLK', 'XLY', 'DBC', 'EEM', 'ZROZ', 
    'IEF', 'XLP', 'XLRE', 'PDBC', 'GSG', 'DBA', 'JJM', 'COPX', 'TIP', 'UUP', 
    'ITB', 'EFA', 'VWO', 'XLC', 'XLV', 'XLF', 'XLI', 'XLB', 'XLE', 'XLRE', 
    'XLP', 'XLY', 'XLK', 'XLU', 'FXE', 'FXY', 'FXB', 'FXC', 'FXA', 'FXF', 
    'CNY', 'CL=F', 'BZ=F', 'NG=F', 'RB=F', 'HO=F', 'GC=F', 'SI=F', 'HG=F', 
    'PL=F', 'PA=F', 'ALI=F', 'ZC=F', 'ZW=F', 'ZS=F', 'KC=F', 'SB=F', 
    'CT=F', 'CC=F', 'LB=F', 'LE=F', 'HE=F', 'GF=F'
]

# Function to download or load data for all assets in parallel
def download_data_parallel(assets):
    df_dict = {}
    failed_tickers = []
    with ThreadPoolExecutor(max_workers=4) as executor:
        future_to_ticker = {executor.submit(get_data, asset): asset for asset in assets}
        for future in as_completed(future_to_ticker):
            asset = future_to_ticker[future]
            try:
                df = future.result()
                if df is not None and 'Adj Close' in df.columns:
                    df_dict[asset] = df['Adj Close']
                else:
                    print(f"Data for {asset} is invalid or missing 'Adj Close', skipping.")
                    failed_tickers.append(asset)
            except Exception as e:
                print(f"Error processing {asset}: {e}")
                failed_tickers.append(asset)
    if failed_tickers:
        print(f"Failed to download data for: {', '.join(failed_tickers)}")
    return df_dict

df_dict = download_data_parallel(all_assets)

# Ensure we have valid data before continuing
if len(df_dict) == 0:
    print("No valid data was downloaded, exiting...")
else:
    # Calculate percentage change for all assets
    pct_change_dict = {}
    for asset, df in df_dict.items():
        if not df.empty:
            pct_change = df.pct_change().dropna()
            if not pct_change.empty:
                pct_change_dict[asset] = pct_change

    # Convert to DataFrame for easier handling
    combined_pct_change_df = pd.DataFrame(pct_change_dict).dropna()

    # Check if the combined DataFrame is still valid
    if combined_pct_change_df.empty:
        print("No valid percentage change data, exiting...")
    else:
        # Save correlation heatmap to a PDF
        with PdfPages(f"{output_folder}/correlation_matrix.pdf") as pdf:
            def plot_correlation_heatmap(combined_pct_change_df):
                corr_matrix = combined_pct_change_df.corr()
                plt.figure(figsize=(16, 12))
                sns.heatmap(corr_matrix, annot=True, cmap='RdBu', center=0, 
                            annot_kws={"size": 8}, fmt='.2f', cbar_kws={"shrink": 0.8})
                plt.xticks(rotation=45, ha="right", fontsize=8)
                plt.yticks(fontsize=8)
                plt.title('Correlation Heatmap of Percentage Changes Across All Assets')
                pdf.savefig()  # Save the heatmap to the PDF
                plt.close()

            plot_correlation_heatmap(combined_pct_change_df)

        print(f"Correlation matrix saved to {output_folder}/correlation_matrix.pdf")

Loading SPY data from data/2024-11-24/SPY.csv
Loading IWM data from data/2024-11-24/IWM.csv
Loading QQQ data from data/2024-11-24/QQQ.csv
Correlation matrix saved to output/2024-11-24/correlation_matrix.pdf


In [25]:
xlu_tickers = [
    "NEE", # NextEra Energy
    "DUK", # Duke Energy
    "SO",  # Southern Company
    "D",   # Dominion Energy
    "AEP", # American Electric Power
    "EXC", # Exelon
    "PEG", # Public Service Enterprise Group
    "ED",  # Consolidated Edison
    "SRE", # Sempra Energy
    "WEC", # WEC Energy Group
]

In [32]:
xlb_tickers = [
    "LIN",  # Linde plc
    "APD",  # Air Products and Chemicals
    "SHW",  # Sherwin-Williams
    "PPG",  # PPG Industries
    "DOW",  # Dow Inc.
    "LYB",  # LyondellBasell Industries
    "ECL",  # Ecolab
    "FCX",  # Freeport-McMoRan
    "NEM",  # Newmont
    "IFF",  # International Flavors & Fragrances
]

In [5]:
xlre_tickers = [
    "PLD",  # Prologis
    "EQIX", # Equinix
    "AMT",  # American Tower
    "CCI",  # Crown Castle International
    "WELL", # Welltower
    "AVB",  # AvalonBay Communities
    "PSA",  # Public Storage
    "SPG",  # Simon Property Group
    "VTR",  # Ventas
    "WY",   # Weyerhaeuser
]

In [6]:
xli_tickers = [
    "UNP",  # Union Pacific
    "HON",  # Honeywell International
    "RTX",  # Raytheon Technologies
    "BA",   # Boeing
    "CAT",  # Caterpillar
    "GE",   # General Electric
    "MMM",  # 3M
    "CSX",  # CSX Corporation
    "DE",   # Deere & Company
    "LMT",  # Lockheed Martin
]

In [7]:
xlv_tickers = [
    "JNJ",  # Johnson & Johnson
    "UNH",  # UnitedHealth Group
    "LLY",  # Eli Lilly and Company
    "MRK",  # Merck & Co.
    "ABBV", # AbbVie
    "PFE",  # Pfizer
    "TMO",  # Thermo Fisher Scientific
    "ABT",  # Abbott Laboratories
    "BMY",  # Bristol-Myers Squibb
    "MDT",  # Medtronic
]

In [22]:
xlf_tickers = [
    "BRK-B", # Berkshire Hathaway
    "JPM",   # JPMorgan Chase
    "BAC",   # Bank of America
    "WFC",   # Wells Fargo
    "BLK",   # BlackRock
    "MS",    # Morgan Stanley
    "GS",    # Goldman Sachs
    "C",     # Citigroup
    "USB",   # U.S. Bancorp
    "PNC",   # PNC Financial Services
]

In [9]:
xlk_tickers = [
    "AAPL",  # Apple
    "MSFT",  # Microsoft
    "NVDA",  # NVIDIA
    "TSM",   # Taiwan Semiconductor
    "META",  # Meta Platforms (formerly Facebook)
    "AVGO",  # Broadcom
    "ADBE",  # Adobe
    "TXN",   # Texas Instruments
    "CRM",   # Salesforce
    "INTC",  # Intel
]

In [45]:
xle_tickers = [
    "LNG",   # Nat Gas
    "XOM",   # Exxon Mobil
    "CVX",   # Chevron
    "COP",   # ConocoPhillips
    "EOG",   # EOG Resources
    "SLB",   # Schlumberger
    "VLO",   # Valero Energy
    "MPC",   # Marathon Petroleum
    "PSX",   # Phillips 66
    "OXY",   # Occidental Petroleum
    "HAL",   # Halliburton
]

In [11]:
xlp_tickers = [
    "PG",    # Procter & Gamble
    "KO",    # Coca-Cola
    "PEP",   # PepsiCo
    "COST",  # Costco Wholesale
    "WMT",   # Walmart
    "MDLZ",  # Mondelez International
    "PM",    # Philip Morris International
    "K",     # Kellogg
    "CL",    # Colgate-Palmolive
    "MO",    # Altria Group
]

In [12]:
xly_tickers = [
    "AMZN", # Amazon
    "TSLA", # Tesla
    "HD",   # Home Depot
    "MCD",  # McDonald's
    "NKE",  # Nike
    "LOW",  # Lowe's
    "SBUX", # Starbucks
    "CMG",  # Chipotle Mexican Grill
    "TJX",  # TJX Companies
    "BKNG", # Booking Holdings
]

In [13]:
xlc_tickers = [
    "GOOGL", # Alphabet (Google) Class A
    "GOOG",  # Alphabet (Google) Class C
    "META",  # Meta Platforms (formerly Facebook)
    "T",     # AT&T
    "CMCSA", # Comcast
    "VZ",    # Verizon
    "TMUS",  # T-Mobile US
    "NFLX",  # Netflix
    "CHTR",  # Charter Communications
    "DIS",   # Walt Disney
]


In [29]:
sectors_list_a = [ "XLU", "XLY","XLP","XLRE","VNQ","KIE","XLF","MOO","XLV","XLI","XLC","XLB","IBB","TAN","KRE","ITB","GDX","XLE","PBW","XLK","XME","SMH"] 

In [52]:
current_assets = xlc_tickers